# Entrenamiento de los datos para modo libre - **Anthony**
---
Para realizar el entrenamiento de la red neuronal para el modo libre, se tomará el set de datos precapturado en el notebook data_collection. Para esto se utilizará pythorch y una red ya pre entrenada de resnet-18.





## Importaciones necesarias

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

## Obtención del set de datos y divición en entrenamiento y test

In [2]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

In [3]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 50, 50]) # 50 imágenes para test y el resto para entrenamiento

In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

## Definición del modelo

---

Utilizando ResNet-18 que es un modelo pre-entrenado en PyTorch TorchVision

In [5]:
model = models.resnet18(pretrained=True)

In [6]:
model.fc = torch.nn.Linear(512, 2)

In [7]:
device = torch.device('cuda')
model = model.to(device)

## Entrenamiento del modelo

---

Se inicia el entrenamiento del modelo a 50 épocas. Se guarda la mejor presición del test.

In [8]:
NUM_EPOCHS = 50
BEST_MODEL_PATH = 'best_model_resnet18.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

print(f"El mejor accuracy fue: {best_accuracy}")

0: 0.880000
1: 0.940000
2: 0.980000
3: 0.940000
4: 0.940000
5: 0.940000
6: 1.000000
7: 0.940000
8: 0.980000
9: 0.980000
10: 1.000000
11: 0.960000
12: 1.000000
13: 0.960000
14: 0.960000
15: 0.960000
16: 1.000000
17: 0.980000
18: 1.000000
19: 1.000000
20: 1.000000
21: 0.940000
22: 0.960000
23: 1.000000
24: 1.000000
25: 1.000000
26: 0.940000
27: 0.980000
28: 0.980000
29: 0.980000
30: 0.960000
31: 0.980000
32: 0.980000
33: 0.960000
34: 0.940000
35: 0.960000
36: 0.960000
37: 0.980000
38: 0.940000
39: 0.980000
40: 1.000000
41: 0.980000
42: 1.000000
43: 0.960000
44: 0.980000
45: 0.960000
46: 0.960000
47: 1.000000
48: 0.940000
49: 0.960000
El mejor accuracy fue: 1.0
